In [41]:
import time
from __future__ import print_function # Use a function definition from future version (say 3.x from 2.7 interpreter)
from PIL import Image
import math
import numpy as np
import os
import PIL
import sys
import cntk as C 
import csv

In [43]:
#da qui prendo sessioni
listaMe2=list()
with open('sessionLabelMe2.csv', 'r') as csvfile:
     spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
     for row in spamreader:
         listaMe2.append(row)

In [44]:
idMe2=list()
labelMe2=np.array([])
for i in range(len(listaMe2)):
    if listaMe2[i][0][13]=='.' and (len(listaMe2[i][0])==20 or len(listaMe2[i][0])==19) :
        idMe2.append(listaMe2[i][0][7:17])
        labelMe2=np.append(labelMe2,int(listaMe2[i][0][18:]))
    if listaMe2[i][0][12]=='.' :
        idMe2.append(listaMe2[i][0][7:16])
        labelMe2=np.append(labelMe2,int(listaMe2[i][0][17:]))

In [45]:
#traccio img bianche
sid3=list()
lab3=np.array([])
for i in range(len(idMe2)):
    if(labelMe2[i]==-1):
        sid3.append(idMe2[i])
        lab3=np.append(lab3,labelMe2[i])

In [47]:
lista=list()
with open('ClassificationNew.csv', 'r') as csvfile:
     spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
     for row in spamreader:
         lista.append(row)

In [49]:
byteArrayFolder="./doveCreoFile"
data_dir="./trackTemp"

In [50]:
sessionId=list()
labels=np.array([])
for i in range(len(lista)-1):
    if lista[i+1][0][2]==',' and lista[i+1][0][7]==',':
        sessionId.append(lista[i+1][0][3:7]+'.png')
        labels=np.append(labels,int(lista[i+1][0][8]))
    elif lista[i+1][0][2]==',' and lista[i+1][0][8]==',':
        sessionId.append(lista[i+1][0][3:8]+'.png')
        labels=np.append(labels,int(lista[i+1][0][9]))
    elif lista[i+1][0][3]==',' and lista[i+1][0][9]==',':
        sessionId.append(lista[i+1][0][4:9]+'.png')
        labels=np.append(labels,int(lista[i+1][0][10]))
    elif lista[i+1][0][4]==',' and lista[i+1][0][10]==',':
        sessionId.append(lista[i+1][0][5:10]+'.png')
        labels=np.append(labels,int(lista[i+1][0][11]))
    elif lista[i+1][0][5]==',' and lista[i+1][0][11]==',':
        sessionId.append(lista[i+1][0][6:11]+'.png')
        labels=np.append(labels,int(lista[i+1][0][12]))
    elif lista[i+1][0][6]==',' and lista[i+1][0][12]==',':
        sessionId.append(lista[i+1][0][7:12]+'.png')
        labels=np.append(labels,int(lista[i+1][0][13]))
    else:#elif lista[i+1][0][6]==',' and lista[i+1][0][13]==',':
        sessionId.append(lista[i+1][0][7:13]+'.png')
        labels=np.append(labels,int(lista[i+1][0][14]))

In [51]:
ctfTrainFile=os.path.join(byteArrayFolder, "trainClass.txt") 
ctfTestFile=os.path.join(byteArrayFolder, "testClass.txt") 

In [52]:
imglist=os.listdir(data_dir)
#imglist=os.listdir(os.path.join("..", "ImgTrack"))
for fichier in imglist[:]: # imgist[:] makes a copy of imglist.
    if not(fichier.endswith(".png")):
        imglist.remove(fichier)

In [53]:
#controllo che ci siano nella cartella
sid=list()
lab=np.array([])
for i in range(len(sessionId)):
    for j in range(len(imglist)):
        if sessionId[i]==imglist[j]:
            sid.append(sessionId[i])
            lab=np.append(lab,labels[i])

In [54]:
#devo togliere img bianche 
conta=0
iVia=np.array([])
for i in range(len(sid)):
    for j in range(len(sid3)):
        if sid[i]==sid3[j]:
            #print(sid[i],lab[i])
            iVia=np.append(iVia,i)
            conta+=1
#print(conta)
sidCopy=sid[:]
labCopy=np.array([])
for u in range(len(iVia)):
    sidCopy.remove(sid[int(iVia[u])])
for t in range(len(sidCopy)):
    for r in range(len(sid)):
        if sidCopy[t]==sid[r]:
            labCopy=np.append(labCopy,lab[r])

In [55]:
len(sid)-len(sidCopy)

1659

In [12]:
sum(labCopy==0),sum(labCopy==1),sum(labCopy==2),sum(labCopy==3)

(4848, 1240, 591, 894)

In [23]:
labs = list(map(' '.join, np.eye(4, dtype=np.uint).astype(str)))
imgWhite = os.path.join(data_dir, '27391.png')
for j in range(len(sidCopy)):
    if lab[j]==0:
        rotationAngle=np.linspace(360//2,360,2)
        for r in range(len(rotationAngle)):
            imgfile = os.path.join(data_dir, sidCopy[j])
            p=PIL.Image.open(imgfile)
            white=PIL.Image.open(imgWhite)
            white=white.resize((150,150))
            white.paste(p,(25,25,125,125),p)
            p_rot=white.rotate(rotationAngle[r])
            p_new=p_rot.crop((5,5,145,145))
            pw=PIL.Image.open(imgWhite)
            pw=pw.resize((140,140))
            pw.paste(p_new,(0,0),p_new)
            p_end=pw.resize((100,100))
            ch=(np.array(p_end)[:,:,0]).flatten()
            ch_lab=np.append(ch,lab[j])
            row_str = ch_lab.astype(str)#qui ci mette lo zero vicino al punto
            label_str = labs[int(ch_lab[-1])]#legge 1.0 come stringa per quello ho messo int
            feature_str = ' '.join(row_str[:-1])
            numRandom=np.random.uniform(0,1)
            if j==0 and r==0:
                with open(ctfTrainFile, 'w') as f:
                    f.write('|labels {} |features {}\n'.format(label_str, feature_str))
                with open(ctfTestFile, 'w') as g:
                    g.write('|labels {} |features {}\n'.format(label_str, feature_str))
            else:   
                if numRandom>0.25:
                    with open(ctfTrainFile, 'a') as f:
                        f.write('|labels {} |features {}\n'.format(label_str, feature_str))
                else:
                    with open(ctfTestFile, 'a') as g:
                        g.write('|labels {} |features {}\n'.format(label_str, feature_str))
    elif lab[j]==1:
        rotationAngle=np.linspace(360//5,360,5)
        for r in range(len(rotationAngle)):
            imgfile = os.path.join(data_dir, sidCopy[j])
            p=PIL.Image.open(imgfile)
            white=PIL.Image.open(imgWhite)
            white=white.resize((150,150))
            white.paste(p,(25,25,125,125),p)
            p_rot=white.rotate(rotationAngle[r])
            p_new=p_rot.crop((5,5,145,145))
            pw=PIL.Image.open(imgWhite)
            pw=pw.resize((140,140))
            pw.paste(p_new,(0,0),p_new)
            p_end=pw.resize((100,100))
            ch=(np.array(p_end)[:,:,0]).flatten()
            ch_lab=np.append(ch,lab[j])
            row_str = ch_lab.astype(str)#qui ci mette lo zero vicino al punto
            label_str = labs[int(ch_lab[-1])]#legge 1.0 come stringa per quello ho messo int
            feature_str = ' '.join(row_str[:-1])
            numRandom=np.random.uniform(0,1)
            if j==0 and r==0:
                with open(ctfTrainFile, 'w') as f:
                    f.write('|labels {} |features {}\n'.format(label_str, feature_str))
                with open(ctfTestFile, 'w') as g:
                    g.write('|labels {} |features {}\n'.format(label_str, feature_str))
            else:   
                if numRandom>0.25:
                    with open(ctfTrainFile, 'a') as f:
                        f.write('|labels {} |features {}\n'.format(label_str, feature_str))
                else:
                    with open(ctfTestFile, 'a') as g:
                        g.write('|labels {} |features {}\n'.format(label_str, feature_str))
    elif lab[j]==2:
        rotationAngle=np.linspace(360//17,360,17)
        for r in range(len(rotationAngle)):
            imgfile = os.path.join(data_dir, sidCopy[j])
            p=PIL.Image.open(imgfile)
            white=PIL.Image.open(imgWhite)
            white=white.resize((150,150))
            white.paste(p,(25,25,125,125),p)
            p_rot=white.rotate(rotationAngle[r])
            p_new=p_rot.crop((5,5,145,145))
            pw=PIL.Image.open(imgWhite)
            pw=pw.resize((140,140))
            pw.paste(p_new,(0,0),p_new)
            p_end=pw.resize((100,100))
            ch=(np.array(p_end)[:,:,0]).flatten()
            ch_lab=np.append(ch,lab[j])
            row_str = ch_lab.astype(str)#qui ci mette lo zero vicino al punto
            label_str = labs[int(ch_lab[-1])]#legge 1.0 come stringa per quello ho messo int
            feature_str = ' '.join(row_str[:-1])
            numRandom=np.random.uniform(0,1)
            if j==0 and r==0:
                with open(ctfTrainFile, 'w') as f:
                    f.write('|labels {} |features {}\n'.format(label_str, feature_str))
                with open(ctfTestFile, 'w') as g:
                    g.write('|labels {} |features {}\n'.format(label_str, feature_str))
            else:   
                if numRandom>0.25:
                    with open(ctfTrainFile, 'a') as f:
                        f.write('|labels {} |features {}\n'.format(label_str, feature_str))
                else:
                    with open(ctfTestFile, 'a') as g:
                        g.write('|labels {} |features {}\n'.format(label_str, feature_str))
    else:
        rotationAngle=np.linspace(360//8,360,8)
        for r in range(len(rotationAngle)):
            imgfile = os.path.join(data_dir, sidCopy[j])
            p=PIL.Image.open(imgfile)
            white=PIL.Image.open(imgWhite)
            white=white.resize((150,150))
            white.paste(p,(25,25,125,125),p)
            p_rot=white.rotate(rotationAngle[r])
            p_new=p_rot.crop((5,5,145,145))
            pw=PIL.Image.open(imgWhite)
            pw=pw.resize((140,140))
            pw.paste(p_new,(0,0),p_new)
            p_end=pw.resize((100,100))
            ch=(np.array(p_end)[:,:,0]).flatten()
            ch_lab=np.append(ch,lab[j])
            row_str = ch_lab.astype(str)#qui ci mette lo zero vicino al punto
            label_str = labs[int(ch_lab[-1])]#legge 1.0 come stringa per quello ho messo int
            feature_str = ' '.join(row_str[:-1])
            numRandom=np.random.uniform(0,1)
            if j==0 and r==0:
                with open(ctfTrainFile, 'w') as f:
                    f.write('|labels {} |features {}\n'.format(label_str, feature_str))
                with open(ctfTestFile, 'w') as g:
                    g.write('|labels {} |features {}\n'.format(label_str, feature_str))
            else:   
                if numRandom>0.25:
                    with open(ctfTrainFile, 'a') as f:
                        f.write('|labels {} |features {}\n'.format(label_str, feature_str))
                else:
                    with open(ctfTestFile, 'a') as g:
                        g.write('|labels {} |features {}\n'.format(label_str, feature_str))

In [24]:
#inizializzazioni variabili input
num_classes = 4
input_dim = 100*100

input_dim_model = (1,100,100)
x = C.input_variable(input_dim_model)
y = C.input_variable(num_classes)

In [25]:
#creazione funzione per lettura del file, tramite il formato ctf ritornando un tipo MinibatchSource
def create_reader(is_training):    
    featureStream = C.io.StreamDef(field='features', shape=input_dim, is_sparse=False)   
    labelStream = C.io.StreamDef(field='labels', shape=num_classes, is_sparse=False)

    streams = C.io.StreamDefs(labels = labelStream, features = featureStream)

    if is_training:
        path = ctfTrainFile
    else:
        path = ctfTestFile
    
    deserializer = C.io.CTFDeserializer(path, streams)         

    mbs = C.io.MinibatchSource(deserializer, randomize = is_training, max_sweeps = C.io.INFINITELY_REPEAT if is_training else 1)
    #se son nel training INFINITELY_REPEAT permettera di coninuare a rigirare nel training all'infinito(decidero io num_minibatch
    #e size), se son nel test voglio che si fermi una volta finiti(con num_minibatch e size) tutti gli elementi del testSet
    return mbs

In [26]:
train = create_reader(True)
test = create_reader(False)

In [27]:
#modello di CNN
def create_model(features):
    with C.layers.default_options(init=C.glorot_uniform(), activation=C.relu):
        h = features
        h = C.layers.Convolution((5,5), 8,strides=(2,2), pad=True,name='first_conv')(h)
        h = C.layers.MaxPooling((10,10), strides=(2,2),name='first_max')(h)
        h = C.layers.Convolution((5,5), 16,strides=(2,2), pad=True,name='second_conv')(h)
        h = C.layers.MaxPooling((10,10), strides=(2,2),name='second_max')(h)
        r = C.layers.Dense(num_classes, activation=None,name='classify')(h)
    return r
#per far saltare fuori cosa succede in mezzo potrei far tornare l'h dopo il primo MaxPooling, modificando questa funzione e la
#sua chiamata a funzione associata

In [28]:
#creazione funzioni di perdita e classficazione errore 
def create_criterion_function(model, labels):
    loss = C.cross_entropy_with_softmax(model, labels)
    errs = C.classification_error(model, labels)
    return loss, errs

In [29]:
#creazione funzione per la stampa, stampa con frequenza prestabilita il numero del minibatch in questione, perdita ed errore
def print_training_progress(trainer, mb, frequency, verbose=1):
    training_loss = "NA"
    eval_error = "NA"
    if mb%frequency == 0:
        training_loss = trainer.previous_minibatch_loss_average
        eval_error = trainer.previous_minibatch_evaluation_average
        if verbose: 
            print ("Minibatch: {0}, Loss: {1:.4f}, Error: {2:.2f}%".format(mb, training_loss, eval_error*100))  
    return mb, training_loss, eval_error 

In [30]:
#normalizzazione dei dati ed impostazione del modello
x_norm=C.element_times(1.0/255.0,x)
z=create_model(x_norm)
#impostazione perdita ed errore
loss, label_error = create_criterion_function(z, y)

In [31]:
#inizializzazione di learning_rate e sua schedule,minibatch_size, momentum, modello di ottimizzazione e numero di minibatch del
#training, circa 10 volte il train(circa 8100 elementi) quindi 8100/64=1265
learning_rate = 0.3
lr_schedule = C.learning_rate_schedule(learning_rate, C.UnitType.minibatch)
minibatch_size =64
momentum_time_constant = C.momentum_as_time_constant_schedule(-minibatch_size/np.log(0.9))
learner = C.adam(z.parameters, lr = lr_schedule, momentum = momentum_time_constant)
num_minibatches_to_train =1265*4#*3

In [32]:
#impostazione trainer
trainer = C.Trainer(z, (loss, label_error), [learner])

In [33]:
#mapping dei dati nelle variaibli d'input
input_map={
        y  : train.streams.labels,
        x  : train.streams.features
    } 

In [34]:
#ciclo in cui si svolge il tutto
for i in range(int(num_minibatches_to_train)):
    data=train.next_minibatch(minibatch_size, input_map=input_map)#return mapping of stream information to MinibatchData
    #Input('Input3', [#], [2 x 100 x 100]): MinibatchData(data=Value([64 x 1 x 20000], la struttura rimane [2x100x100] ma con i
    #dati dentro mappati
    trainer.train_minibatch(data)
    print_training_progress(trainer, i, 250, verbose=1)

Minibatch: 0, Loss: 1.3823, Error: 60.94%
Minibatch: 250, Loss: 1.3149, Error: 60.94%
Minibatch: 500, Loss: 1.2647, Error: 56.25%
Minibatch: 750, Loss: 1.2181, Error: 57.81%
Minibatch: 1000, Loss: 1.3077, Error: 64.06%
Minibatch: 1250, Loss: 1.1691, Error: 56.25%
Minibatch: 1500, Loss: 1.3216, Error: 60.94%
Minibatch: 1750, Loss: 1.1799, Error: 53.12%
Minibatch: 2000, Loss: 1.2763, Error: 54.69%
Minibatch: 2250, Loss: 1.1228, Error: 50.00%
Minibatch: 2500, Loss: 1.0487, Error: 42.19%
Minibatch: 2750, Loss: 1.2045, Error: 57.81%
Minibatch: 3000, Loss: 1.2082, Error: 54.69%
Minibatch: 3250, Loss: 1.1654, Error: 51.56%
Minibatch: 3500, Loss: 1.3100, Error: 60.94%
Minibatch: 3750, Loss: 1.0835, Error: 50.00%
Minibatch: 4000, Loss: 1.1940, Error: 53.12%
Minibatch: 4250, Loss: 1.1529, Error: 57.81%
Minibatch: 4500, Loss: 1.1810, Error: 56.25%
Minibatch: 4750, Loss: 1.1397, Error: 59.38%
Minibatch: 5000, Loss: 1.1494, Error: 56.25%


In [35]:
#mapping per il test
input_map = {
        y  : test.streams.labels,
        x  : test.streams.features
}

In [36]:
#circa 2700 elementi nel test, per star sicuro ne guardo 2500 facendo 25 minibatch di taglia 100
test_minibatch_size =100
num_samples=7500
num_minibatches_to_test =num_samples // test_minibatch_size
test_result = 0.0   

In [37]:
#valuto testSet e calcolo errore
for i in range(num_minibatches_to_test):
    data = test.next_minibatch(test_minibatch_size, input_map=input_map)
    eval_error = trainer.test_minibatch(data)
    test_result = test_result + eval_error 
print("Average test error: {0:.2f}%".format(test_result*100 / num_minibatches_to_test)) 

Average test error: 49.24%


solo label 0,1

In [3]:
ctfTrainFile=os.path.join(byteArrayFolder, "trainClass01.txt") 
ctfTestFile=os.path.join(byteArrayFolder, "testClass01.txt") 

In [14]:
labs = list(map(' '.join, np.eye(2, dtype=np.uint).astype(str)))
imgWhite = os.path.join(data_dir, '27391.png')
for j in range(len(sidCopy)):
    if lab[j]==0:
        rotationAngle=np.linspace(360//2,360,2)
        for r in range(len(rotationAngle)):
            imgfile = os.path.join(data_dir, sidCopy[j])
            p=PIL.Image.open(imgfile)
            white=PIL.Image.open(imgWhite)
            white=white.resize((150,150))
            white.paste(p,(25,25,125,125),p)
            p_rot=white.rotate(rotationAngle[r])
            p_new=p_rot.crop((5,5,145,145))
            pw=PIL.Image.open(imgWhite)
            pw=pw.resize((140,140))
            pw.paste(p_new,(0,0),p_new)
            p_end=pw.resize((100,100))
            ch=(np.array(p_end)[:,:,0]).flatten()
            ch_lab=np.append(ch,lab[j])
            row_str = ch_lab.astype(str)#qui ci mette lo zero vicino al punto
            label_str = labs[int(ch_lab[-1])]#legge 1.0 come stringa per quello ho messo int
            feature_str = ' '.join(row_str[:-1])
            numRandom=np.random.uniform(0,1)
            if j==0 and r==0:
                with open(ctfTrainFile, 'w') as f:
                    f.write('|labels {} |features {}\n'.format(label_str, feature_str))
                with open(ctfTestFile, 'w') as g:
                    g.write('|labels {} |features {}\n'.format(label_str, feature_str))
            else:   
                if numRandom>0.25:
                    with open(ctfTrainFile, 'a') as f:
                        f.write('|labels {} |features {}\n'.format(label_str, feature_str))
                else:
                    with open(ctfTestFile, 'a') as g:
                        g.write('|labels {} |features {}\n'.format(label_str, feature_str))
    elif lab[j]==1:
        rotationAngle=np.linspace(360//5,360,5)
        for r in range(len(rotationAngle)):
            imgfile = os.path.join(data_dir, sidCopy[j])
            p=PIL.Image.open(imgfile)
            white=PIL.Image.open(imgWhite)
            white=white.resize((150,150))
            white.paste(p,(25,25,125,125),p)
            p_rot=white.rotate(rotationAngle[r])
            p_new=p_rot.crop((5,5,145,145))
            pw=PIL.Image.open(imgWhite)
            pw=pw.resize((140,140))
            pw.paste(p_new,(0,0),p_new)
            p_end=pw.resize((100,100))
            ch=(np.array(p_end)[:,:,0]).flatten()
            ch_lab=np.append(ch,lab[j])
            row_str = ch_lab.astype(str)#qui ci mette lo zero vicino al punto
            label_str = labs[int(ch_lab[-1])]#legge 1.0 come stringa per quello ho messo int
            feature_str = ' '.join(row_str[:-1])
            numRandom=np.random.uniform(0,1)
            if j==0 and r==0:
                with open(ctfTrainFile, 'w') as f:
                    f.write('|labels {} |features {}\n'.format(label_str, feature_str))
                with open(ctfTestFile, 'w') as g:
                    g.write('|labels {} |features {}\n'.format(label_str, feature_str))
            else:   
                if numRandom>0.25:
                    with open(ctfTrainFile, 'a') as f:
                        f.write('|labels {} |features {}\n'.format(label_str, feature_str))
                else:
                    with open(ctfTestFile, 'a') as g:
                        g.write('|labels {} |features {}\n'.format(label_str, feature_str))

In [5]:
#inizializzazioni variabili input
num_classes = 2
input_dim = 100*100

input_dim_model = (1,100,100)
x = C.input_variable(input_dim_model)
y = C.input_variable(num_classes)

In [6]:
#creazione funzione per lettura del file, tramite il formato ctf ritornando un tipo MinibatchSource
def create_reader(is_training):    
    featureStream = C.io.StreamDef(field='features', shape=input_dim, is_sparse=False)   
    labelStream = C.io.StreamDef(field='labels', shape=num_classes, is_sparse=False)

    streams = C.io.StreamDefs(labels = labelStream, features = featureStream)

    if is_training:
        path = ctfTrainFile
    else:
        path = ctfTestFile
    
    deserializer = C.io.CTFDeserializer(path, streams)         

    mbs = C.io.MinibatchSource(deserializer, randomize = is_training, max_sweeps = C.io.INFINITELY_REPEAT if is_training else 1)
    #se son nel training INFINITELY_REPEAT permettera di coninuare a rigirare nel training all'infinito(decidero io num_minibatch
    #e size), se son nel test voglio che si fermi una volta finiti(con num_minibatch e size) tutti gli elementi del testSet
    return mbs

In [7]:
train = create_reader(True)
test = create_reader(False)

In [7]:
#due modelli scelti alla fine del tuning
def create_model(features):
    with C.layers.default_options(init=C.glorot_uniform(), activation=C.relu):
        h = features
        h = C.layers.Convolution((3,3), 8,strides=(2,2), pad=True,name='first_conv')(h)
        h = C.layers.MaxPooling((2,2), strides=(2,2),name='first_max')(h)
        h = C.layers.Convolution((3,3), 16,strides=(2,2), pad=True,name='second_conv')(h)
        h = C.layers.MaxPooling((2,2), strides=(2,2),name='second_max')(h)
        h = C.layers.Dense(16)(h)
        r = C.layers.Dense(num_classes, activation=None,name='classify')(h)
    return r

In [8]:
def create_model(features):
    with C.layers.default_options(init=C.glorot_uniform(), activation=C.relu):
        h = features
        h = C.layers.Convolution((5,5), 8,strides=(1,1), pad=True,name='first_conv')(h)
        h = C.layers.MaxPooling((2,2), strides=(2,2),name='first_max')(h)
        h = C.layers.Convolution((5,5), 16,strides=(1,1), pad=True,name='second_conv')(h)
        h = C.layers.MaxPooling((2,2), strides=(2,2),name='second_max')(h)
        h = C.layers.Convolution((5,5), 32,strides=(1,1), pad=True,name='third_conv')(h)
        h = C.layers.MaxPooling((2,2), strides=(2,2),name='third_max')(h)
        h = C.layers.Convolution((5,5), 64,strides=(1,1), pad=True,name='fourth_conv')(h)
        h = C.layers.MaxPooling((2,2), strides=(2,2),name='fourth_max')(h)
        h = C.layers.Convolution((5,5), 128,strides=(1,1), pad=True,name='fifth_conv')(h)
        h = C.layers.MaxPooling((2,2), strides=(2,2),name='fifth_max')(h)
        h = C.layers.Dropout(0.5)(h)
        r = C.layers.Dense(num_classes, activation=None,name='classify')(h)
    return r

In [9]:
#creazione funzioni di perdita e classficazione errore 
def create_criterion_function(model, labels):
    loss = C.cross_entropy_with_softmax(model, labels)
    errs = C.classification_error(model, labels)
    return loss, errs

In [10]:
#creazione funzione per la stampa, stampa con frequenza prestabilita il numero del minibatch in questione, perdita ed errore
def print_training_progress(trainer, mb, frequency, verbose=1):
    training_loss = "NA"
    eval_error = "NA"
    if mb%frequency == 0:
        training_loss = trainer.previous_minibatch_loss_average
        eval_error = trainer.previous_minibatch_evaluation_average
        if verbose: 
            print ("Minibatch: {0}, Loss: {1:.4f}, Error: {2:.2f}%".format(mb, training_loss, eval_error*100))  
    return mb, training_loss, eval_error 

In [11]:
#normalizzazione dei dati ed impostazione del modello
x_norm=C.element_times(1.0/256.0,x)
z=create_model(x_norm)
#impostazione perdita ed errore
loss, label_error = create_criterion_function(z, y)

In [12]:
#inizializzazione di learning_rate e sua schedule,minibatch_size, momentum, modello di ottimizzazione e numero di minibatch del
#training, circa 10 volte il train(circa 8100 elementi) quindi 8100/64=1265
learning_rate = 0.1
lr_schedule = C.learning_rate_schedule(learning_rate, C.UnitType.minibatch)
minibatch_size =64
momentum_time_constant = C.momentum_as_time_constant_schedule(-minibatch_size/np.log(0.9))
learner = C.adam(z.parameters, lr = lr_schedule, momentum = momentum_time_constant)
num_minibatches_to_train =150000/minibatch_size

In [13]:
#impostazione trainer
trainer = C.Trainer(z, (loss, label_error), [learner])

In [14]:
#mapping dei dati nelle variaibli d'input
input_map={
        y  : train.streams.labels,
        x  : train.streams.features
    } 

In [15]:
#ciclo in cui si svolge il tutto
for i in range(int(num_minibatches_to_train)):
    data=train.next_minibatch(minibatch_size, input_map=input_map)#return mapping of stream information to MinibatchData
    #Input('Input3', [#], [2 x 100 x 100]): MinibatchData(data=Value([64 x 1 x 20000], la struttura rimane [2x100x100] ma con i
    #dati dentro mappati
    trainer.train_minibatch(data)
    print_training_progress(trainer, i, num_minibatches_to_train//10, verbose=1)

Minibatch: 0, Loss: 0.6864, Error: 39.06%
Minibatch: 234, Loss: 0.6180, Error: 35.94%
Minibatch: 468, Loss: 0.2889, Error: 12.50%
Minibatch: 702, Loss: 0.3227, Error: 14.06%
Minibatch: 936, Loss: 0.4890, Error: 17.19%
Minibatch: 1170, Loss: 0.2665, Error: 14.06%
Minibatch: 1404, Loss: 0.2515, Error: 10.94%
Minibatch: 1638, Loss: 0.2491, Error: 14.06%
Minibatch: 1872, Loss: 0.1631, Error: 6.25%
Minibatch: 2106, Loss: 0.2274, Error: 7.81%
Minibatch: 2340, Loss: 0.3679, Error: 14.06%


In [16]:
#mapping per il test
input_map = {
        y  : test.streams.labels,
        x  : test.streams.features
}

In [17]:
#circa 2700 elementi nel test, per star sicuro ne guardo 2500 facendo 25 minibatch di taglia 100
test_minibatch_size =100
num_samples=4000
num_minibatches_to_test =num_samples // test_minibatch_size
test_result = 0.0  

In [18]:
#valuto testSet e calcolo errore
for i in range(num_minibatches_to_test):
    data = test.next_minibatch(test_minibatch_size, input_map=input_map)
    eval_error = trainer.test_minibatch(data)
    test_result = test_result + eval_error 
print("Average test error: {0:.2f}%".format(test_result*100 / num_minibatches_to_test)) 

Average test error: 9.40%
